##  原始数据格式数据使用numpy.loadtxt读取

In [175]:
from numpy import loadtxt
tmp = loadtxt('../data/fm_train_real.dat').T

## libsvm格式数据使用svmutil.svm_read_problem读取，这个最难用了

In [395]:
import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from svmutil import *
from svm import *
test = svm_read_problem('/home/bd-dev/lijian/201801_ICML/tmp/iris.scale')

##  libsvm格式数据使用sklearn带的方法读取，这个好用

In [394]:
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

@mem.cache
def get_data():
    data = load_svmlight_file("/home/bd-dev/lijian/201801_ICML/tmp/iris.scale")
    return data[0], data[1]

X, y = get_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## libsvm格式数据使用shogun带的方法读取，这个一般 

In [389]:
from tempfile import NamedTemporaryFile
from shogun import SparseRealFeatures
from shogun import LibSVMFile
fname = '/home/bd-dev/lijian/201801_ICML/tmp/iris.scale'
f=SparseRealFeatures()
lab=f.load_with_labels(LibSVMFile(fname))
tmp_file = NamedTemporaryFile(suffix='svmlight')
f.save_with_labels(LibSVMFile(tmp_file.name, 'w'), lab)
f.d
print type(f)
print type(lab)

<type 'shogun.SparseRealFeatures'>
<type 'numpy.ndarray'>


## .mat数据格式使用scipy.io.load读取

In [9]:
import scipy.io as sio
data = sio.loadmat('/home/bd-dev/lijian/201801_ICML/tmp/RawData/plant/plant.phylpro.mat');
data['phylpros'].shape
X=data['phylpros']

data=sio.loadmat('/home/bd-dev/lijian/201801_ICML/data/plant/label_plant.mat');
y=data['y'];

## 分片->生成核矩阵
### 样本交叉验证时维度为(n,d)，生成核矩阵时维度为(d,n)。标签的数据格式是float64，且标签维度:(n,1)->(n,)

In [15]:
print X_train.T.shape
print y_train.shape

(89, 752)
(752, 1)


In [11]:
from shogun import RealFeatures, MulticlassLabels
from shogun import GaussianKernel
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

width=2.1
C=1
epsilon=1e-5

feats_train=RealFeatures(X_train.T)
feats_test=RealFeatures(X_test.T)
kernel=GaussianKernel(feats_train, feats_train, width)

labels=MulticlassLabels(y_train.reshape(y_train.size,).astype('float64'))

classifier = LibSVM()
classifier.set_epsilon(epsilon)
#print labels.get_labels()
mc_classifier = KernelMulticlassMachine(MulticlassOneVsRestStrategy(),kernel,classifier,labels)
mc_classifier.train()

kernel.init(feats_train, feats_test)
out = mc_classifier.apply().get_labels()

## 整理成方法

In [ ]:
#!/usr/bin/env python

import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import scipy.io as sio
import numpy as np
from svmutil import *
from svm import *
from shogun import RealFeatures, MulticlassLabels
from shogun import GaussianKernel
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from sklearn.model_selection import train_test_split

# OneVsRest learning machine
def classifier_multiclassmachine (fm_train_real,fm_test_real,label_train_multiclass,width=2.1,C=1,epsilon=1e-5):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    kernel=GaussianKernel(feats_train, feats_train, width)

    labels=MulticlassLabels(label_train_multiclass)

    classifier = LibSVM()
    classifier.set_epsilon(epsilon)
    #print labels.get_labels()
    mc_classifier = KernelMulticlassMachine(C, MulticlassOneVsRestStrategy(),kernel,classifier,labels)
    mc_classifier.train()

    kernel.init(feats_train, feats_test)
    out = mc_classifier.apply().get_labels()

    return out

# load data from txt
def loadFromTxt (file_path):
    return np.loadtxt(file_path)

# load data from libsvm data
def loadFromLibsvm (file_path):
    sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
    return svm_read_problem(file_path)

# load data from .mat file
def loadFromLibsvm (file_path):
    return sio.loadmat(file_path)

# phylpro type file        
def FinalResult4phylpro(data_name):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=test_size)
        label_pre = classifier_multiclassmachine(X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), width, C, epsilon)
        accuracy.append((y_test.reshape(y_test.size,).astype('float64')==label_pre).sum()/float(label_pre.size))
        print 'finish round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print("\n".join(str(item) for item in accuracy))

@mem.cache
# svmlight type file        
def FinalResult4svmlight(data_name):
    print 'cj'
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()

    accuracy=[]
    for i in range(times):        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        label_pre = classifier_multiclassmachine(X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), width, C, epsilon)
        accuracy.append((y_test.reshape(y_test.size,).astype('float64')==label_pre).sum()/float(label_pre.size))
        print 'finish round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print("\n".join(str(item) for item in accuracy))

## 运行方法的实例

In [6]:
if __name__=='__main__':
    times=30
    C=100
    test_size=0.2
    width=8
    epsilon=1e-5
    FinalResult4svmlight('iris')    
    print 'test'

test
